In [1]:
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [76]:
driver = webdriver.Chrome()             
url = "https://www.wunderground.com/history"
driver.get(url)                                                         #abrimos la url
driver.maximize_window()                                                #maximizamos la ventana                                             
driver.implicitly_wait(20)                                              #durante 20 segundo va a estar intentandi hacer click abajo pero si lo hace ante pues hace click

##excepcional si las coockies estan en otra capa

iframe = WebDriverWait(driver,10).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))        #duerme dirante 10segundo y le doy una condicion
driver.switch_to.frame(iframe)
try:
    driver.find_element('css selector','#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button').click()
except:
    print('No encuentro el boton')

driver.switch_to.default_content()                                      #vuelce a la primera pagina
sleep(5)
driver.find_element('css selector','#historySearch').send_keys('Madrid,Madrid,España',Keys.ENTER)
sleep(2)
driver.find_element('xpath','//*[@id="dateSubmit"]').click()
driver.find_element('xpath','//*[@id="dateSubmit"]').click()
sleep(2)
mount = driver.find_element('xpath','//*[@id="station-select-button"]').click()
sleep(2)
mount2 = driver.find_element('css selector','#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)').click()
sleep(3)
driver.execute_script('window.scrollBy(0, 2000)')
sleep(3)
html_page = driver.page_source
sopa = BeautifulSoup(html_page) 


In [123]:
sopa
lista_tablas = sopa.findAll("table")
len(lista_tablas)

9

In [142]:
lista_tablas_final = lista_tablas[1]
lista_tablas_final
enunciados = [enunciado.getText() for enunciado in lista_tablas_final.findAll("thead")]
enunciados2 = enunciados[0].split()


In [148]:
df = pd.DataFrame(columns=enunciados2)
df
                                                    #tenemos las columns

,TimeTemperature,(°F)Dew,Point,(°F)Humidity,(%)Wind,Speed,(mph)Pressure,(in)Precipitation,(in)


In [179]:
time = lista_tablas[2]
temperature = lista_tablas[3]
dewpoint = lista_tablas[4]
humidity = lista_tablas[5]
windspeed= lista_tablas[6]
pressure = lista_tablas[7]
precipitation = lista_tablas[8]

In [216]:
tabla = [tiempo.getText() for tiempo in time.findAll('td')]
nombre =[tabla[0].strip()]
columna1 = tabla[1:]
df_columas1 = pd.DataFrame(columna1, columns=nombre)
df_columas1.head(2)


,Oct
0,1
1,2


In [218]:
tabla1= [t.getText() for t in temperature.findAll('td')]
columnamaxymin = tabla1[:3]
datos11 = tabla1[3:]
reshaped_data = np.array(datos11).reshape(-1, 3)
df1 = pd.DataFrame(reshaped_data, columns=columnamaxymin)
df1.head(2)

,Max,Avg,Min
0,84,66.8,50
1,79,71.2,63


In [219]:
tabla2= [d.getText() for d in dewpoint.findAll('td')]
columnamaxymin2 = tabla2[:3]
datos2 = tabla2[3:]
reshaped2 = np.array(datos2).reshape(-1, 3)
df2 = pd.DataFrame(reshaped2, columns=columnamaxymin2 )
df2.head(2)

,Max,Avg,Min
0,55,45.2,41
1,63,57.0,52


In [220]:
tabla3= [h.getText() for h in humidity.findAll('td')]
columnasmaxim3 = tabla3[:3]
datos3 = tabla3[3:]
reshaped3 = np.array(datos3).reshape(-1, 3)
df3 = pd.DataFrame(reshaped3, columns=columnasmaxim3)
df3.head(2)


,Max,Avg,Min
0,72,48.8,23
1,78,62.0,47


In [222]:
tabla4= [ws.getText() for ws in windspeed.findAll('td')]
columna4 = tabla4[:3]
datos4 = tabla4[3:]
reshaped4 = np.array(datos4).reshape(-1, 3)
df4 = pd.DataFrame(reshaped4, columns=columna4)
df4.head(2
         )


,Max,Avg,Min
0,10,3.6,0
1,18,8.8,0


In [225]:
tabla5= [ps.getText() for ps in pressure.findAll('td')]
columna5 = tabla5[:3]
data5 = tabla5[3:]
reshaped5 = np.array(data5).reshape(-1, 3)
df5 = pd.DataFrame(reshaped5, columns=columna5)
df5.head(2)

,Max,Avg,Min
0,28.0,27.9,27.9
1,27.9,27.8,27.7


In [228]:
tabla6= [precip.getText() for precip in precipitation('td')]
name = [tabla6[0].strip()]
values = tabla6[1:]
df6= pd.DataFrame(values, columns=name)

In [233]:
df_final = pd.concat([df_columas1,df1, df2, df3, df4, df5,df6 ], axis=1)
df_final

,Oct,Max,Avg,Min,Max,Avg,Min,Max,Avg,Min,Max,Avg,Min,Max,Avg,Min,Total
0,1,84,66.8,50,55,45.2,41,72,48.8,23,10,3.6,0,28.0,27.9,27.9,0.00
1,2,79,71.2,63,63,57.0,52,78,62.0,47,18,8.8,0,27.9,27.8,27.7,0.00
2,3,77,67.6,61,59,49.6,37,88,55.5,24,16,5.6,0,27.9,27.8,27.7,0.00
3,4,77,63.9,50,50,46.1,43,82,53.9,36,8,2.3,0,27.9,27.9,27.8,0.00
4,5,81,65.5,50,59,51.0,45,87,61.8,39,17,5.9,0,27.9,27.9,27.8,0.00
5,6,75,69.7,64,59,56.7,55,73,64.1,50,17,12.7,1,27.8,27.8,27.8,0.00
6,7,75,63.6,57,57,53.6,48,94,71.8,38,24,6.9,0,27.8,27.7,27.6,0.00
7,8,64,60.0,55,57,54.6,52,88,82.0,68,17,11.4,3,27.8,27.8,27.7,0.00
8,9,70,63.0,59,63,57.3,50,94,81.4,63,23,12.2,3,27.8,27.6,27.5,0.00
9,10,68,60.4,52,50,43.9,41,76,55.4,37,20,7.3,0,27.9,27.8,27.7,0.00
